In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
net = cv2.dnn.readNet("yolov4-tiny_final.weights", "yolov4-tiny.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

In [ ]:
def yolo_detect(img):
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    boxes = []
    confidences = []
    class_ids = []
    
    for out in outs:
        for detection in out:
            tx, ty, tw, th, confidence = detection[0:5]
            scores = detection[5:]
            class_id = np.argmax(scores)
            if confidence>0.5:
                center_x = int(tx*width)
                center_y = int(ty*height)
                w = int(tw*width)
                h = int(th*height)
                x = int(center_x-w/2)
                y = int(center_y-h/2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.5)
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            crop_img = img[y:y+h, x:x+w]
            
    return crop_img

In [ ]:
from tensorflow.keras.models import load_model
cnn = load_model('cnn_car_number.h5')

In [ ]:
# 0-9的列表
label_name_09 = [str(i) for i in range(10)]
print(label_name_09)

# A-Z的列表
label_name_AZ = [chr(i+65) for i in range(26)]
print(label_name_AZ)

# 串接
label_name = label_name_09 + label_name_AZ

In [ ]:
def cnn_number(img):
    sh, sw, _ = img.shape
    gray_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    _, binary_img = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY_INV)

    save_predict_name = []
    save_contours = []
    contours, hierarchy = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for i in range(len(contours)):
        (x, y, w, h) = cv2.boundingRect(contours[i]) 
        save_contours.append((x, y, w, h))
    save_contours = sorted(save_contours, key=lambda x:x[0])
            
    for (x, y, w, h) in save_contours:
        if sw*(3/20)>w>sw*(0.5/20) and sh*(19.5/20)>h>sh*(12/20):
            number = gray_img[y:y+h, x:x+w]
            re_number = cv2.resize(number, (50, 50),interpolation=cv2.INTER_CUBIC) 
            intput_number = np.expand_dims(np.expand_dims(re_number.astype('float32')/255,-1),0) 
            predict = cnn.predict(intput_number)
            predict_name = label_name[np.argmax(predict)]
            save_predict_name.append(predict_name) 
    
    return "".join(save_predict_name)

In [ ]:
img = cv2.imread("bmpraw001.bmp")
crop_img = yolo_detect(img)

plt.imshow(crop_img)
plt.show()

In [ ]:
cnn_number(crop_img)